In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('./bertweet').to('cuda')
tokenizer = AutoTokenizer.from_pretrained('./bertweet')

In [ ]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=0)

In [ ]:
def get_predictions(text_list):
    inputs = tokenizer(text_list, padding=True, truncation=True, return_tensors='pt').to('cuda')
    outputs = model(**inputs)
    probs = outputs[0].softmax(1)
    labels = [prob.argmax().item() for prob in probs] # 0:biden 1:trump
    return labels

In [ ]:
import re
import json
from tqdm import tqdm

biden_0, trump_1, batch_size, text_list = 0, 0, 32, []
for i, line in tqdm(enumerate(open('202004.txt', encoding='utf-8'))):
    text_list.append(line)
    if len(text_list) == batch_size:      
        labels = get_predictions(text_list)
        summation = sum(labels)
        trump_1 += summation
        biden_0 += batch_size - summation
        text_list = []

print('biden:', biden_0)
print('trump:', trump_1)